# Trasfer Learning classificazione e regressione segnali stradali

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import os
import keras as ks
from keras.preprocessing import image as kimage
import skimage.io as io
import random
import sys
import pickle as pkl
from imblearn.keras import BalancedBatchGenerator
from keras.utils import np_utils
lib_path = os.path.join(os.path.realpath(r"/content/drive/My Drive/multi_task/DataAugmentationForObjectDetection-master"), "data_aug")
sys.path.append(lib_path)
from bbox_util import *
from data_aug import *
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import Model
from keras.layers import Concatenate,Input, Dense, Flatten, Reshape, MaxPooling2D, Conv2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dense, Dropout, BatchNormalization, LeakyReLU
from keras.optimizers import SGD
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, explained_variance_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils import shuffle
import skimage as sk
from skimage import transform
from skimage import util
from keras import backend as K

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should 

## Import dei dati

In [0]:
X = np.load('/content/drive/My Drive/multi_task/X.npy')
Y = np.load('/content/drive/My Drive/multi_task/Y.npy')
X_test = np.load('/content/drive/My Drive/multi_task/X_test.npy')
y_test = np.load('/content/drive/My Drive/multi_task/y_test.npy')

## Preprocessing

In [0]:
opt = ks.optimizers.Nadam(lr=0.0001, beta_1=0.9, beta_2=0.999, schedule_decay=0.01)
opt2 = SGD(lr=0.001, momentum=0.9, nesterov=True, decay=0.001) # lr = 0.01, decay=0.0001
opt3 = ks.optimizers.RMSprop(lr=0.001, rho=0.9)
opt4 = ks.optimizers.Adagrad(lr=0.01)
opt5 = ks.optimizers.Adadelta(lr= 1.0, rho=0.95)
opt6 = ks.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, amsgrad=False)
opt7 = ks.optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999)

In [0]:
Y = Y[:, 1:] #visto che sono tutti segnali, elimino la prima colonna (quella che dice se c'è o meno il segnale)    
y_test = y_test[:, 1:]

In [0]:
def random_noise(image_array):
    # add random noise to the image
    return sk.util.random_noise(image_array, var = 0.001)

In [0]:
def augmentation(immagini, y):
    inter = [cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_LANCZOS4]
    for i in range(0, immagini.shape[0]):
        down = cv2.resize(immagini[i], (30,30), interpolation = random.choice(inter))
        immagini[i] = cv2.resize(down, (40,40))
        immagini[i] = random_noise(immagini[i])
    return immagini, y

In [0]:
X_train, y_train = shuffle(X,Y)

In [0]:
#X_train, y_train = augmentation(X_train, y_train)

# Funzioni Utili

In [0]:
def new_draw_rect(img, bboxes):
    x1 = bboxes[0]
    y1 = bboxes[1]
    x2 = bboxes[2]
    y2 = bboxes[3]
    new_bb = np.array([[x1, y1, x2, y2, 0]])
    plt.imshow(draw_rect(img, new_bb))

In [0]:
def find_area(arr):
    width = arr[2] - arr[0]
    height = arr[1] - arr[3]
    area = width * height
    return area

In [0]:
def run(bboxes1, bboxes2):
    #https://github.com/venuktan/Intersection-Over-Union/blob/master/iou_benchmark.py
    x11, y11, x12, y12 = tf.split(bboxes1, 4, axis=1)
    x21, y21, x22, y22 = tf.split(bboxes2, 4, axis=1)
    xA = tf.maximum(x11, tf.transpose(x21))
    yA = tf.maximum(y11, tf.transpose(y21))
    xB = tf.minimum(x12, tf.transpose(x22))
    yB = tf.minimum(y12, tf.transpose(y22))
    interArea = tf.maximum((xB - xA + 1), 0) * tf.maximum((yB - yA + 1), 0)
    boxAArea = (x12 - x11 + 1) * (y12 - y11 + 1)
    boxBArea = (x22 - x21 + 1) * (y22 - y21 + 1)
    iou = interArea / ((boxAArea + tf.transpose(boxBArea) - interArea)+1)
    return iou

In [0]:
def get_iou(bb1, bb2):
    """
    #https://stackoverflow.com/questions/25349178/calculating-percentage-of-bounding-box-overlap-for-image-detector-evaluation
    Calculate the Intersection over Union (IoU) of two bounding boxes.
    
    Parameters
    ----------
    bb1 : dict
        np.array (['x1', 'x2', 'y1', 'y2'])
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        np.array (['x1', 'x2', 'y1', 'y2'])
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1[0] < bb1[2]
    assert bb1[1] < bb1[3]
    assert bb2[0] < bb2[2]
    assert bb2[1] < bb2[3]
        # determine the coordinates of the intersection rectangle
    x_left = max(bb1[0], bb2[0])
    y_top = max(bb1[1], bb2[1])
    x_right = min(bb1[2], bb2[2])
    y_bottom = min(bb1[3], bb2[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0
    
    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1[2] - bb1[0]) * (bb1[3] - bb1[1])
    bb2_area = (bb2[2] - bb2[0]) * (bb2[3] - bb2[1])
    
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [0]:
def iou_loss(y_true, y_pred):
    iou = run(y_true, y_pred)
    return 1/(iou + 1) #compreso tra 1/2 e 1

In [0]:
def custom_balance(immagini, segnale, bbox):
    rus = RandomUnderSampler(return_indices=True)
    X_resampled, y_resampled, indexes = rus.fit_sample(immagini.reshape(immagini.shape[0], immagini.shape[1]*immagini.shape[2]*immagini.shape[3]), np.argmax(segnale, axis = 1))     
    X_resampled = X_resampled.reshape(X_resampled.shape[0], immagini.shape[1], immagini.shape[2], immagini.shape[3])
    y_resampled = np_utils.to_categorical(y_resampled, num_classes=43)
    #y_resampled = np.concatenate([y_resampled, bbox[indexes]], axis = 1)
    return(X_resampled, y_resampled[:,0:43])# y_resampled[:,43:]]

In [0]:
def balanced_flow_from_directory(train_gen):
    while(True):
        immagini, y = next(train_gen)
        yield custom_balance(immagini, y[:,0:43], y[:,43:])

## TL MODELLO

In [0]:
from keras.preprocessing import image as kimage
from keras.preprocessing import image as kimage
from keras.applications.resnet import  ResNet50
from keras.applications import resnet
from keras.applications import resnet_v2
from keras.applications import ResNet50V2
train_img_pro = kimage.ImageDataGenerator(preprocessing_function=resnet_v2.preprocess_input)

In [0]:
train_generator_flow = train_img_pro.flow((X_train, y_train), 
                    batch_size=1500,
                    shuffle=True #riordino i dati in maniera casuale
                    )      
train_generator_bal = balanced_flow_from_directory(train_generator_flow)

In [0]:
base_net = ResNet50V2(input_shape=(40,40,3),
                         include_top=False, #ignorare l'ultimo fully connected
                         weights='imagenet', #pesi ottimizzati su imagenet
                         pooling='avg')

In [0]:
for layer in base_net.layers:
  layer.trainable = False #Blocchiamo tutti i livelli della rete e non alleniamoli

In [0]:
#base_net = Model(model.input, model.output) #[-9]

In [0]:
#base_net.summary()

In [0]:
base_net = Model(base_net.input, base_net.layers[-30].output) #[-9]
x = base_net.output
#x = Conv2D(filters = 512, strides=1, kernel_size=1)(x)
x = Flatten()(x)
x = Dense(1024, activation = 'relu')(x)
x = Dense(512, activation = 'relu')(x)
x_1 = Dense(256, activation = 'relu')(x)
#x_1 = Dropout(0.3)(x_1)
output1 = Dense(43, activation = 'softmax', name = 'classificazione') (x_1)
#x_2 = Dense(256, activation = 'relu')(x)
#x_2 = Dropout(0.3)(x_2)
#output2 = Dense(4, activation='linear', name = 'regressione')(x_2)
net = Model(inputs=base_net.input, outputs = output1)

In [0]:
opt = ks.optimizers.Nadam(lr=0.0001, beta_1=0.9, beta_2=0.999, schedule_decay=0.01)
opt2 = SGD(lr=0.001, momentum=0.9, nesterov=True, decay=0.001) # lr = 0.01, decay=0.0001
opt3 = ks.optimizers.RMSprop(lr=0.001, rho=0.9)
opt4 = ks.optimizers.Adagrad(lr=0.01)
opt5 = ks.optimizers.Adadelta(lr= 1.0, rho=0.95)
opt6 = ks.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, amsgrad=False)
opt7 = ks.optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999)

In [0]:
net.compile(loss = 'categorical_crossentropy', optimizer=opt2, metrics = {'classificazione':'accuracy'}) 

In [0]:
#net.save_weights("/content/drive/My Drive/Pesi_modelli/pesi_class_segnali.hdf5")

In [0]:
from keras.callbacks import History
import warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
history = History()
net.fit_generator(train_generator_bal, epochs = 200, callbacks=[history], steps_per_epoch= 100) #150

In [0]:
#net.save_weights("/content/drive/My Drive/Pesi_modelli/pesi_class_segnali.hdf5")

In [0]:
prova = resnet_v2.preprocess_input(X_train)

In [0]:
prediz = net.predict(prova)

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train[:,:43].argmax(axis = -1),prediz.argmax(axis = -1))

0.38994873625953225

In [0]:
diz = history.history

In [0]:
import json
#with open('/content/drive/My Drive/Pesi_modelli/history1.txt', 'w') as file:
     #file.write(json.dumps(diz)) # use `json.loads` to do the reverse

In [0]:
import os
import skimage.io as io
from PIL import Image
import re
X = []
Y = []
conteggio = []
i = 0 
path = r"/content/drive/My Drive/Dataset/dataset_classificazione_multiclasse_creato_da_noi/"
Class=os.listdir(path)
for a in Class:
    new_path = path+a
    class_2 = os.listdir(new_path)
    for j in class_2: 
        image = io.imread(path+a+"/"+j)
        imResize = cv2.resize(image,(40,40))
        #image_from_array = Image.fromarray(image, 'RGB')
        X.append(np.asarray(imResize))
        Y.append(int(re.search('(^\d+)',a).group(1)))
      #train = np.concatenate([train,np.expand_dims(np.asarray(image), axis = 0)], axis = 0)
      #train_labels = np.concatenate([train_labels,np.array([a])], axis = 0)

In [0]:
X_inference = resnet_v2.preprocess_input(np.array(X))
y_inference = np_utils.to_categorical(Y)

In [0]:
prediction = net.predict(X_inference)

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(y_inference.argmax(axis = -1),prediction.argmax(axis = -1))

0.3225806451612903